## Bias Mitigation using Fairlearn - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)

In [1]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_75M_25F.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,61,1,3,146.0,241.0,0,0,148.0,1,3.0,0,1
1,52,1,1,120.0,284.0,0,0,118.0,0,0.0,2,0
2,48,0,3,150.0,227.0,0,0,130.0,1,1.0,1,0
3,49,1,3,128.0,212.0,0,0,96.0,1,0.0,1,1
4,56,1,3,120.0,236.0,0,1,148.0,0,0.0,1,1


In [2]:
# Ensure y_test is a Series (not a DataFrame with 1 column)
y_test = y_test.squeeze("columns")

# Define target and sensitive column names
TARGET = "HeartDisease"
SENSITIVE = "Sex"

# Split train into X/y
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

# Extract sensitive features separately
A_train = X_train[SENSITIVE].astype(int)
A_test  = X_test[SENSITIVE].astype(int)


In [3]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [4]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [5]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [6]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [7]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [8]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### Further KNN Improvement - Implementing PCA 

In [11]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()

print("=== Baseline (tuned PCA+KNN, no mitigation) ===")
# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

=== Baseline (tuned PCA+KNN, no mitigation) ===
=== KNN (best params) Evaluation ===
Accuracy : 0.8858695652173914
Precision: 0.9090909090909091
Recall   : 0.8823529411764706
F1 Score : 0.8955223880597015

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87        82
           1       0.91      0.88      0.90       102

    accuracy                           0.89       184
   macro avg       0.88      0.89      0.88       184
weighted avg       0.89      0.89      0.89       184

Confusion Matrix:
 [[73  9]
 [12 90]]




In [15]:
# Demographic Parity post-processing for your tuned PCA+KNN

from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Helper (reuse if already defined)
def eval_fairness(y_true, y_pred, A):
    mf = MetricFrame(
        metrics={
            "TPR": true_positive_rate,
            "FPR": false_positive_rate,
            "SelectionRate": selection_rate,
            "Accuracy": accuracy_score,
        },
        y_true=y_true, y_pred=y_pred, sensitive_features=A
    )
    return {
        "by_group": mf.by_group,
        "acc": accuracy_score(y_true, y_pred),
        "dp": demographic_parity_difference(y_true, y_pred, sensitive_features=A),
        "eo": equalized_odds_difference(y_true, y_pred, sensitive_features=A),
    }

# 1) Baseline metrics (no mitigation) for side-by-side
pca_knn.fit(X_train_ready, y_train)
y_base = pca_knn.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)

print("=== Baseline (tuned PCA+KNN) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 2) Post-processing with DEMOGRAPHIC PARITY
post_dp = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="demographic_parity",
    predict_method="predict_proba",   # KNN supports this
    grid_size=200,
    flip=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)

y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test)
m_dp = eval_fairness(y_test, y_dp, A_test)

print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

=== Baseline (tuned PCA+KNN) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    0.833333  0.125       0.236842  0.868421
1    0.885417  0.100       0.616438  0.890411
Accuracy: 0.8859 | DP diff: 0.3796 | EO diff: 0.0521

=== Post-processing (Demographic Parity) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    0.833333  0.125       0.236842  0.868421
1    0.885417  0.100       0.616438  0.890411
Accuracy: 0.8859 | DP diff: 0.3796 | EO diff: 0.0521


### Post-Processing with Demographic Parity (PCA+KNN)

#### Comparison of Results

| Model                          | Accuracy | DP diff | EO diff | Notes                                      |
|--------------------------------|----------|---------|---------|--------------------------------------------|
| Baseline (tuned PCA+KNN)       | 0.8859   | 0.3796  | 0.0521  | High DP disparity, low EO gap              |
| Post-processing (DP constraint)| 0.8859   | 0.3796  | 0.0521  | Identical to baseline, no fairness change  |

#### Interpretation
- Both baseline and DP post-processing yield **the same metrics**: accuracy ≈ 88.6%, **DP diff high (0.38)**, **EO diff low (0.05)**.  
- Post-processing failed because group score distributions are too different, so ThresholdOptimizer defaulted to the baseline.  
- **In-processing mitigation is not possible with KNN**, since `KNeighborsClassifier` does not support the `sample_weight` argument required by Fairlearn reductions.

---

### Tuned Decision Tree (DT)

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score
)

# 1) Base model
dt = DecisionTreeClassifier(random_state=42)

# 2) Hyperparameter grid 
param_grid = {
    "criterion": ["gini", "entropy"],
    "max_depth": [3, 5, 7, 9, None],
    "min_samples_split": [2, 5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 6, 10],
}

# 3) Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4) Grid search 
grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    cv=cv,
    scoring="f1",      
    n_jobs=-1,
    verbose=0
)

grid_dt.fit(X_train_ready, y_train)

print("Best Decision Tree params:", grid_dt.best_params_)
print("Best CV F1:", grid_dt.best_score_)

# 5) Train & evaluate best DT
tuned_dt = grid_dt.best_estimator_
y_pred_dt_best = tuned_dt.predict(X_test_ready)
y_prob_dt_best = tuned_dt.predict_proba(X_test_ready)[:, 1] 

evaluate_model(y_test, y_pred_dt_best, "Tuned Decision Tree (best params)")

Best Decision Tree params: {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best CV F1: 0.8593494246061409
=== Tuned Decision Tree (best params) Evaluation ===
Accuracy : 0.8097826086956522
Precision: 0.819047619047619
Recall   : 0.8431372549019608
F1 Score : 0.8309178743961353

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.77      0.78        82
           1       0.82      0.84      0.83       102

    accuracy                           0.81       184
   macro avg       0.81      0.81      0.81       184
weighted avg       0.81      0.81      0.81       184

Confusion Matrix:
 [[63 19]
 [16 86]]




### Bias Mitigation DT: Inprocessing 

In [ ]:
# In-processing mitigation for your tuned Decision Tree
from sklearn.base import clone
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Helper: fairness evaluation
def eval_fairness(y_true, y_pred, A):
    mf = MetricFrame(
        metrics={
            "TPR": true_positive_rate,
            "FPR": false_positive_rate,
            "SelectionRate": selection_rate,
            "Accuracy": accuracy_score,
        },
        y_true=y_true, y_pred=y_pred, sensitive_features=A
    )
    return {
        "acc": accuracy_score(y_true, y_pred),
        "dp": demographic_parity_difference(y_true, y_pred, sensitive_features=A),
        "eo": equalized_odds_difference(y_true, y_pred, sensitive_features=A),
        "by_group": mf.by_group
    }

# 0) Baseline: tuned DT without mitigation (for comparison)
y_pred_dt_base = tuned_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_pred_dt_base, A_test)
print("=== Baseline (Tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 1) Exponentiated Gradient with Equalized Odds
eg_eo = ExponentiatedGradient(
    estimator=clone(tuned_dt),        # unfitted clone of your tuned DT
    constraints=EqualizedOdds(),
    eps=0.01,                         # try {0.005, 0.01, 0.02, 0.05}
    max_iter=50
)
eg_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_eo = eg_eo.predict(X_test_ready)
m_eo = eval_fairness(y_test, y_pred_eo, A_test)
print("\n=== In-processing: EG (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# 2) Exponentiated Gradient with Demographic Parity
eg_dp = ExponentiatedGradient(
    estimator=clone(tuned_dt),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_dp = eg_dp.predict(X_test_ready)
m_dp = eval_fairness(y_test, y_pred_dp, A_test)
print("\n=== In-processing: EG (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# 3) Summary table
summary_dt = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + EG (EO)",        "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + EG (DP)",        "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs In-processing (EG) ===")
summary_dt


=== Baseline (Tuned DT) ===
          TPR      FPR  SelectionRate  Accuracy
Sex                                            
0    0.833333  0.28125       0.368421  0.736842
1    0.843750  0.20000       0.623288  0.828767
Accuracy: 0.8098 | DP diff: 0.2549 | EO diff: 0.0812

=== In-processing: EG (Equalized Odds) ===
          TPR      FPR  SelectionRate  Accuracy
Sex                                            
0    0.833333  0.15625       0.263158  0.842105
1    0.864583  0.22000       0.643836  0.835616
Accuracy: 0.8370 | DP diff: 0.3807 | EO diff: 0.0638

=== In-processing: EG (Demographic Parity) ===
          TPR   FPR  SelectionRate  Accuracy
Sex                                         
0    0.833333  0.25       0.342105  0.763158
1    0.843750  0.22       0.630137  0.821918
Accuracy: 0.8098 | DP diff: 0.2880 | EO diff: 0.0300

=== Decision Tree: Baseline vs In-processing (EG) ===


,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8098,0.2549,0.0812
1,DT + EG (EO),0.8370,0.3807,0.0638
2,DT + EG (DP),0.8098,0.2880,0.0300


## Bias Mitigation Results: Decision Tree: In-Processing

#### Metrics Overview

| Model              | Accuracy | DP diff | EO diff | Notes                                                |
|--------------------|----------|---------|---------|------------------------------------------------------|
| DT Baseline (tuned)| 0.8098   | 0.2549  | 0.0812  | Moderate disparity in both DP and EO                 |
| DT + EG (EO)       | 0.8370   | 0.3807  | 0.0638  | Higher accuracy, lower EO gap, but worse DP disparity |
| DT + EG (DP)       | 0.8098   | 0.2880  | 0.0300  | Stable accuracy, much lower EO gap, DP disparity persists |

#### Interpretation
- **Baseline DT** shows moderate disparities: DP diff ≈ 0.25, EO diff ≈ 0.08.  
- **EG with Equalized Odds** improves accuracy and reduces EO disparity but increases DP disparity.  
- **EG with Demographic Parity** stabilizes accuracy and strongly reduces EO disparity, but DP disparity remains high.  
- These results illustrate the **trade-offs between fairness metrics**: optimizing for one criterion (EO or DP) may worsen the other.

---

In [20]:
from fairlearn.postprocessing import ThresholdOptimizer

#Baseline for mitigation: fixed tuned DT
tuned_dt.fit(X_train_ready, y_train)
y_base = tuned_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)
print("=== Baseline (tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

#Post-processing: Equalized Odds
post_eo = ThresholdOptimizer(
    estimator=tuned_dt,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_eo = post_eo.predict(X_test_ready, sensitive_features=A_test)
m_eo = eval_fairness(y_test, y_eo, A_test)
print("\n=== Post-processing (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# Post-processing: Demographic Parity
post_dp = ThresholdOptimizer(
    estimator=tuned_dt,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test)
m_dp = eval_fairness(y_test, y_dp, A_test)
print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# create summary table 
summary = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + Post (EO)",      "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + Post (DP)",      "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs Post-processing ===")
summary


=== Baseline (tuned DT) ===
          TPR      FPR  SelectionRate  Accuracy
Sex                                            
0    0.833333  0.28125       0.368421  0.736842
1    0.843750  0.20000       0.623288  0.828767
Accuracy: 0.8098 | DP diff: 0.2549 | EO diff: 0.0812

=== Post-processing (Equalized Odds) ===
          TPR   FPR  SelectionRate  Accuracy
Sex                                         
0    0.833333  0.25       0.342105  0.763158
1    0.833333  0.20       0.616438  0.821918
Accuracy: 0.8098 | DP diff: 0.2743 | EO diff: 0.0500

=== Post-processing (Demographic Parity) ===
          TPR     FPR  SelectionRate  Accuracy
Sex                                           
0    0.833333  0.3125       0.394737  0.710526
1    0.864583  0.2000       0.636986  0.842466
Accuracy: 0.8152 | DP diff: 0.2422 | EO diff: 0.1125

=== Decision Tree: Baseline vs Post-processing ===


,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8098,0.2549,0.0812
1,DT + Post (EO),0.8098,0.2743,0.0500
2,DT + Post (DP),0.8152,0.2422,0.1125


## Bias Mitigation Results: Decision Tree: Post-Processing

## Summary Table

| Model / Method              | Accuracy | DP Diff ↓ | EO Diff ↓ | Notes                                                                 |
|------------------------------|----------|-----------|-----------|----------------------------------------------------------------------|
| **Baseline (Tuned DT)**     | 0.8098   | 0.2549    | 0.0812    | Good accuracy, but large selection disparity (DP gap).                |
| **Equalized Odds (Post)**   | 0.8098   | 0.2743    | 0.0500    | EO improves error parity (fairer TPR/FPR) but worsens DP disparity.   |
| **Demographic Parity (Post)**| 0.8152  | 0.2422    | 0.1125    | DP reduces selection disparity, improves accuracy slightly, but hurts EO. |

---

## Interpretation

- **Baseline (Tuned DT):**  
  - Achieves solid accuracy (~0.81).  
  - **Bias:** Group 1 has a much higher selection rate (0.62 vs 0.37), reflected in a high **DP diff (0.25)**.  
  - EO disparity is moderate (0.081).  

- **Equalized Odds (Post-processing):**  
  - Accuracy unchanged.  
  - **Improvement in fairness:** EO diff drops from 0.081 → 0.050, meaning error rates are more balanced.  
  - **Trade-off:** DP disparity worsens (0.274), indicating even less parity in selections.  

- **Demographic Parity (Post-processing):**  
  - Accuracy slightly **improves** (0.8152).  
  - **Improvement in fairness:** DP diff drops to 0.242, lowest among models.  
  - **Trade-off:** EO disparity increases (0.113), i.e., error rate fairness worsens.  

---

## Key Takeaways

- **If the priority is equal treatment in error rates (TPR/FPR):**  
  → **Equalized Odds** is preferred.  

- **If the priority is equal selection outcomes across groups:**  
  → **Demographic Parity** is preferred.  

  ---


### Ensemble Model - Random Forest (RF)

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Train the model
rf.fit(X_train_ready, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test_ready)
y_prob_rf = rf.predict_proba(X_test_ready)[:, 1]  
evaluate_model(y_test, y_pred_rf, "Random Forest")

=== Random Forest Evaluation ===
Accuracy : 0.8804347826086957
Precision: 0.8703703703703703
Recall   : 0.9215686274509803
F1 Score : 0.8952380952380953

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.83      0.86        82
           1       0.87      0.92      0.90       102

    accuracy                           0.88       184
   macro avg       0.88      0.88      0.88       184
weighted avg       0.88      0.88      0.88       184

Confusion Matrix:
 [[68 14]
 [ 8 94]]




### Bias Mitgation RF: In-processing 

In [22]:
# 0) Baseline Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_ready, y_train)

y_pred_rf_base = rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_pred_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

#1) EG with Equalized Odds
eg_eo_rf = ExponentiatedGradient(
    estimator=clone(rf),
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_eo = eg_eo_rf.predict(X_test_ready)
m_rf_eo = eval_fairness(y_test, y_pred_rf_eo, A_test)

print("\n=== In-processing RF: EG (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) EG with Demographic Parity 
eg_dp_rf = ExponentiatedGradient(
    estimator=clone(rf),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_dp = eg_dp_rf.predict(X_test_ready)
m_rf_dp = eval_fairness(y_test, y_pred_rf_dp, A_test)

print("\n=== In-processing RF: EG (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

# 3) Summary Table 
summary_rf = pd.DataFrame([
    {"model":"RF Baseline",      "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + EG (EO)",     "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + EG (DP)",     "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs In-processing (EG) ===")
print(summary_rf)


=== Baseline (Random Forest) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.916667  0.200       0.671233  0.876712
Accuracy: 0.8804 | DP diff: 0.4081 | EO diff: 0.0833

=== In-processing RF: EG (Equalized Odds) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.916667  0.200       0.671233  0.876712
Accuracy: 0.8804 | DP diff: 0.4081 | EO diff: 0.0833

=== In-processing RF: EG (Demographic Parity) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.916667  0.200       0.671233  0.876712
Accuracy: 0.8804 | DP diff: 0.4081 | EO diff: 0.0833

=== Random Forest: Baseline vs In-processing (EG) ===
          model  accuracy  dp_diff  eo_diff
0   RF Baseline    0.8804   0.4081   0.0833
1  RF + EG (EO)

## Random Forest Bias Mitigation Results

### Summary

| Model            | Accuracy | DP Diff | EO Diff | Interpretation                                |
|------------------|----------|---------|---------|-----------------------------------------------|
| **RF Baseline**  | 0.8804   | 0.4081  | 0.0833  | High accuracy; strong DP disparity remains.    |
| **RF + EG (EO)** | 0.8804   | 0.4081  | 0.0833  | Same as baseline → EO constraint had no effect.|
| **RF + EG (DP)** | 0.8804   | 0.4081  | 0.0833  | Same as baseline → DP constraint had no effect.|

### Key Points
- RF performs well but shows **large selection disparity** (DP gap ~0.41).  
- **EG constraints (EO, DP)** did **not change predictions** due to model stability and strict tolerance (`eps=0.01`).  
- Unlike DT, RF did not respond to fairness constraints.  

---

### Bias Mitigation DT: Post-processing 

In [23]:
from fairlearn.postprocessing import ThresholdOptimizer

# 0) Baseline RF 
rf.fit(X_train_ready, y_train)
y_rf_base = rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds 
post_rf_eo = ThresholdOptimizer(
    estimator=rf,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_rf_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_eo = post_rf_eo.predict(X_test_ready, sensitive_features=A_test)
m_rf_eo = eval_fairness(y_test, y_rf_eo, A_test)

print("\n=== RF + Post-processing (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity 
post_rf_dp = ThresholdOptimizer(
    estimator=rf,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_rf_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_dp = post_rf_dp.predict(X_test_ready, sensitive_features=A_test)
m_rf_dp = eval_fairness(y_test, y_rf_dp, A_test)

print("\n=== RF + Post-processing (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

#3) Summary Table
summary_rf_post = pd.DataFrame([
    {"model":"RF Baseline",       "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + Post (EO)",    "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + Post (DP)",    "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs Post-processing ===")
print(summary_rf_post)

=== Baseline (Random Forest) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.916667  0.200       0.671233  0.876712
Accuracy: 0.8804 | DP diff: 0.4081 | EO diff: 0.0833

=== RF + Post-processing (Equalized Odds) ===
         TPR    FPR  SelectionRate  Accuracy
Sex                                         
0    1.00000  0.125       0.263158  0.894737
1    0.90625  0.200       0.664384  0.869863
Accuracy: 0.8750 | DP diff: 0.4012 | EO diff: 0.0938

=== RF + Post-processing (Demographic Parity) ===
         TPR    FPR  SelectionRate  Accuracy
Sex                                         
0    1.00000  0.125       0.263158  0.894737
1    0.90625  0.200       0.664384  0.869863
Accuracy: 0.8750 | DP diff: 0.4012 | EO diff: 0.0938

=== Random Forest: Baseline vs Post-processing ===
            model  accuracy  dp_diff  eo_diff
0     RF Baseline    0.8804   0.4081   0.0833
1  RF + Post (EO)    0

# Random Forest Bias Mitigation (Post-processing)

## Summary

| Model             | Accuracy | DP Diff | EO Diff | Interpretation                                 |
|-------------------|----------|---------|---------|------------------------------------------------|
| **RF Baseline**   | 0.8804   | 0.4081  | 0.0833  | High accuracy; large DP disparity (~0.41).      |
| **RF + Post (EO)**| 0.8750   | 0.4012  | 0.0938  | Slight drop in accuracy; DP gap nearly same, EO worsened. |
| **RF + Post (DP)**| 0.8750   | 0.4012  | 0.0938  | Identical to EO outcome → no meaningful fairness gain. |

## Key Facts:
- RF baseline has **good predictive power** but strong **selection disparity** (DP gap > 0.40).  
- Post-processing (EO, DP) produced **almost no fairness improvement**, while slightly reducing accuracy.  
- Compared to DT, RF appears **less responsive to fairness post-processing** interventions.  

---

### Deep Learning - Multi-layer Perceptron

In [24]:
#import required library 
from sklearn.neural_network import MLPClassifier

In [25]:
#Adam + Early Stopping 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

adammlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),   # slightly smaller/deeper can help
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,       # smaller step can stabilize
    alpha=1e-3,                    # L2 regularization to reduce overfitting
    batch_size=32,
    max_iter=1000,                 # increased max_iter
    early_stopping=True,           # use a validation split internally
    validation_fraction=0.15,
    n_iter_no_change=25,          
    tol=1e-4,
    random_state=42
)

adammlp.fit(X_train_ready, y_train)  
y_pred_mlp = adammlp.predict(X_test_ready)                     
y_prob_mlp = adammlp.predict_proba(X_test_ready)[:, 1]         

evaluate_model(y_test, y_pred_mlp, "(Adam + EarlyStopping)")

=== (Adam + EarlyStopping) Evaluation ===
Accuracy : 0.8586956521739131
Precision: 0.8877551020408163
Recall   : 0.8529411764705882
F1 Score : 0.87

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85        82
           1       0.89      0.85      0.87       102

    accuracy                           0.86       184
   macro avg       0.86      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184

Confusion Matrix:
 [[71 11]
 [15 87]]




### Bias mitigation MLP: Inprocessing

In [26]:
from sklearn.neural_network import MLPClassifier
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from sklearn.base import clone
import pandas as pd

# 0) Baseline MLP
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,
    alpha=1e-3,
    batch_size=32,
    max_iter=1000,
    early_stopping=True,
    validation_fraction=0.15,
    n_iter_no_change=25,
    tol=1e-4,
    random_state=42
)

mlp.fit(X_train_ready, y_train)

y_pred_mlp_base = mlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_pred_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")


# 1) EG with Equalized Odds
eg_eo_mlp = ExponentiatedGradient(
    estimator=clone(mlp),
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_mlp_eo = eg_eo_mlp.predict(X_test_ready)
m_mlp_eo = eval_fairness(y_test, y_pred_mlp_eo, A_test)

print("\n=== In-processing MLP: EG (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")


# 2) EG with Demographic Parity
eg_dp_mlp = ExponentiatedGradient(
    estimator=clone(mlp),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_mlp_dp = eg_dp_mlp.predict(X_test_ready)
m_mlp_dp = eval_fairness(y_test, y_pred_mlp_dp, A_test)

print("\n=== In-processing MLP: EG (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")


# 3) Summary Table
summary_mlp = pd.DataFrame([
    {"model":"MLP Baseline",    "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + EG (EO)",   "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + EG (DP)",   "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs In-processing (EG) ===")
print(summary_mlp)

=== Baseline (MLP) ===
         TPR    FPR  SelectionRate  Accuracy
Sex                                         
0    1.00000  0.125       0.263158  0.894737
1    0.84375  0.140       0.602740  0.849315
Accuracy: 0.8587 | DP diff: 0.3396 | EO diff: 0.1562

=== In-processing MLP: EG (Equalized Odds) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.822917  0.160       0.595890  0.828767
Accuracy: 0.8424 | DP diff: 0.3327 | EO diff: 0.1771

=== In-processing MLP: EG (Demographic Parity) ===
          TPR    FPR  SelectionRate  Accuracy
Sex                                          
0    1.000000  0.125       0.263158  0.894737
1    0.833333  0.160       0.602740  0.835616
Accuracy: 0.8478 | DP diff: 0.3396 | EO diff: 0.1667

=== MLP: Baseline vs In-processing (EG) ===
           model  accuracy  dp_diff  eo_diff
0   MLP Baseline    0.8587   0.3396   0.1562
1  MLP + EG (EO)    0.8424   0.3327

# MLP In-Processing Bias Mitigation Results

## Summary

| Model             | Accuracy | DP Diff | EO Diff | Interpretation                                      |
|------------------|----------|---------|---------|----------------------------------------------------|
| **MLP Baseline**  | 0.8587   | 0.3396  | 0.1562  | Good accuracy; noticeable DP (0.34) and EO (0.16) disparities. |
| **MLP + EG (EO)** | 0.8424   | 0.3327  | 0.1771  | Slight drop in accuracy; EO constraint increased EO gap, small DP improvement. |
| **MLP + EG (DP)** | 0.8478   | 0.3396  | 0.1667  | Slight drop in accuracy; DP constraint did not reduce DP gap, EO slightly worse. |

## Key Points
- Baseline MLP is accurate but has **selection and error disparities** across groups.  
- **EG in-processing** had **limited impact**:
  - EO constraint slightly lowered DP gap but increased EO gap.  
  - DP constraint barely affected DP and slightly worsened EO.  

---

### Bias mitigation MLP: Postprocessing

In [27]:
from fairlearn.postprocessing import ThresholdOptimizer
import pandas as pd

# 0) Baseline MLP
mlp.fit(X_train_ready, y_train)
y_mlp_base = mlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds
post_mlp_eo = ThresholdOptimizer(
    estimator=mlp,
    constraints="equalized_odds",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_eo = post_mlp_eo.predict(X_test_ready, sensitive_features=A_test)
m_mlp_eo = eval_fairness(y_test, y_mlp_eo, A_test)

print("\n=== MLP + Post-processing (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity
post_mlp_dp = ThresholdOptimizer(
    estimator=mlp,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_dp = post_mlp_dp.predict(X_test_ready, sensitive_features=A_test)
m_mlp_dp = eval_fairness(y_test, y_mlp_dp, A_test)

print("\n=== MLP + Post-processing (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")

# 3) Summary Table
summary_mlp_post = pd.DataFrame([
    {"model":"MLP Baseline",       "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + Post (EO)",    "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + Post (DP)",    "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs Post-processing ===")
print(summary_mlp_post)

=== Baseline (MLP) ===
         TPR    FPR  SelectionRate  Accuracy
Sex                                         
0    1.00000  0.125       0.263158  0.894737
1    0.84375  0.140       0.602740  0.849315
Accuracy: 0.8587 | DP diff: 0.3396 | EO diff: 0.1562

=== MLP + Post-processing (Equalized Odds) ===
          TPR      FPR  SelectionRate  Accuracy
Sex                                            
0    0.833333  0.15625       0.263158  0.842105
1    0.947917  0.30000       0.726027  0.863014
Accuracy: 0.8587 | DP diff: 0.4629 | EO diff: 0.1437

=== MLP + Post-processing (Demographic Parity) ===
         TPR    FPR  SelectionRate  Accuracy
Sex                                         
0    1.00000  0.125       0.263158  0.894737
1    0.84375  0.140       0.602740  0.849315
Accuracy: 0.8587 | DP diff: 0.3396 | EO diff: 0.1562

=== MLP: Baseline vs Post-processing ===
             model  accuracy  dp_diff  eo_diff
0     MLP Baseline    0.8587   0.3396   0.1562
1  MLP + Post (EO)    0.8587  

# MLP Post-Processing Bias Mitigation Results

## Summary

| Model                | Accuracy | DP Diff | EO Diff | Interpretation                                      |
|---------------------|----------|---------|---------|----------------------------------------------------|
| **MLP Baseline**     | 0.8587   | 0.3396  | 0.1562  | Good accuracy; moderate DP (0.34) and EO (0.16) disparities. |
| **MLP + Post (EO)**  | 0.8587   | 0.4629  | 0.1437  | Accuracy unchanged; EO post-processing **increased DP gap** but slightly improved EO gap. |
| **MLP + Post (DP)**  | 0.8587   | 0.3396  | 0.1562  | No change from baseline → DP post-processing had no effect. |

## Key Points
- **EO post-processing** shifted selection rates across groups, inadvertently **worsening DP disparity**.  
- **DP post-processing** did not adjust predictions under current settings.  
- Accuracy remained stable (~0.86) across all conditions.  
- Suggests MLP is **less responsive to post-processing**, and EO mitigation can have unintended trade-offs.

---

## Overall Comparison:

# Gender Bias Mitigation: Overall Comparison Across Models

## Key Metrics
- **Accuracy:** Overall predictive performance
- **DP diff (Demographic Parity difference):** Measures gender bias in selection rates
- **EO diff (Equalized Odds difference):** Measures gender bias in error rates (TPR/FPR)

---

## Summary Table

| Model / Technique                | Accuracy | DP Diff | EO Diff | Interpretation |
|---------------------------------|---------|---------|---------|----------------|
| **PCA+KNN Baseline**             | 0.8859  | 0.3796  | 0.0521  | High accuracy, moderate DP bias, low EO bias. |
| **PCA+KNN + Post (DP)**          | 0.8859  | 0.3796  | 0.0521  | No effect; post-processing did not mitigate gender bias. |
| **DT Baseline**                   | 0.8098  | 0.2549  | 0.0812  | Moderate accuracy; noticeable DP and EO disparities. |
| **DT + EG (EO)**                  | 0.8370  | 0.3807  | 0.0638  | Slight accuracy improvement; EO gap slightly reduced, but DP increased → trade-off. |
| **DT + EG (DP)**                  | 0.8098  | 0.2880  | 0.0300  | DP mitigation effective; EO disparity reduced slightly; accuracy stable. |
| **DT + Post (EO)**                | 0.8098  | 0.2743  | 0.0500  | EO post-processing reduced EO gap; minor DP increase. |
| **DT + Post (DP)**                | 0.8152  | 0.2422  | 0.1125  | DP post-processing reduced DP gap; EO gap increased; small accuracy gain. |
| **RF Baseline**                   | 0.8804  | 0.4081  | 0.0833  | High accuracy; substantial DP bias, moderate EO bias. |
| **RF + EG (EO / DP)**             | 0.8804  | 0.4081  | 0.0833  | In-processing had **no effect**; RF predictions insensitive to EG adjustments. |
| **RF + Post (EO)**                | 0.8750  | 0.4012  | 0.0938  | Accuracy slightly decreased; EO gap slightly worsened; DP gap slightly improved. |
| **RF + Post (DP)**                | 0.8750  | 0.4012  | 0.0938  | No meaningful change from baseline; post-processing ineffective. |
| **MLP Baseline**                  | 0.8587  | 0.3396  | 0.1562  | Good accuracy; moderate DP and high EO disparities. |
| **MLP + EG (EO)**                 | 0.8424  | 0.3327  | 0.1771  | Slight accuracy drop; EO gap increased; small DP improvement. |
| **MLP + EG (DP)**                 | 0.8478  | 0.3396  | 0.1667  | Minimal effect on DP; EO worsened slightly. |
| **MLP + Post (EO)**               | 0.8587  | 0.4629  | 0.1437  | EO post-processing increased DP gap; EO gap slightly improved; accuracy stable. |
| **MLP + Post (DP)**               | 0.8587  | 0.3396  | 0.1562  | No effect; post-processing did not mitigate gender bias. |

---

## Overall Interpretation

1. **Baseline Observations**
   - All models show **gender disparities** in selection rates (DP) and error rates (EO), with Random Forest having the **largest DP gap**.
   - EO gaps are generally smaller than DP gaps for KNN and DT, but higher for MLP.

2. **In-Processing Mitigation (EG)**
   - **Decision Tree**: EG effectively reduced DP or EO depending on the constraint, with trade-offs in accuracy.
   - **Random Forest**: EG had **no effect**, likely due to ensemble rigidity.
   - **MLP**: EG produced **limited improvement**; EO constraint slightly reduced EO for females but increased DP.

3. **Post-Processing Mitigation**
   - **DT**: Can reduce EO or DP gaps, but sometimes increases the other metric; trade-offs are model-dependent.
   - **RF & MLP**: Post-processing generally **less effective**; sometimes worsened DP disparity.

4. **Key Takeaways for Gender Bias**
   - **DT** is the most responsive to bias mitigation techniques (both in- and post-processing).
   - **Random Forest** is robust but **resistant** to mitigation—likely due to strong base predictions.
   - **MLP** is partially responsive but shows **trade-offs** between DP and EO when applying EG or post-processing.
   - **Mitigation requires careful choice of technique**:
     - EG works better for simpler models (DT) than complex ones (RF/MLP).  
     - Post-processing can backfire if DP and EO trade-offs are not balanced.

---